## AUTHENTICATION

In [1]:
from nubra_python_sdk.start_sdk import InitNubraSdk, NubraEnv
from auth import authenticate_nubra

# For UAT
# nubra = authenticate_nubra(NubraEnv.UAT)
# For PROD
nubra = authenticate_nubra(NubraEnv.PROD)
# nubra = InitNubraSdk(NubraEnv.PROD)

🔑 Enter your MPIN:  8080


In [3]:
nubra = authenticate_nubra(NubraEnv.UAT)

## REFERENCE_ID_FETCHER

In [6]:
# data/websocket_client.py

# data/websocket_client.py

from nubra_python_sdk.marketdata.market_data import MarketData
from nubra_python_sdk.refdata.instruments import InstrumentData



def fetch_instrument_ref_ids(nubra_sdk, nubra_names):
    """
    Given a list of Nubra instrument names,
    fetches and returns a list of instrument reference IDs.
    
    Parameters
    ----------
    nubra_sdk : InitNubraSdk instance (UAT)
    nubra_names : list[str]
        List of Nubra instrument names
    
    Returns
    -------
    list[int]
        List of instrument ref_id values
    """

    market_data = MarketData(nubra_sdk)
    instruments = InstrumentData(nubra_sdk)

    ref_ids = []

    for name in nubra_names:
        instrument = instruments.get_instrument_by_nubra_name(name)
        ref_ids.append(instrument.ref_id)

    return ref_ids


In [8]:
# # data/websocket_client.py

# from nubra_python_sdk.refdata.instruments import InstrumentData


# def fetch_instrument_ref_ids(nubra_sdk, nubra_names):
#     """
#     Given a list of Nubra instrument names,
#     fetches and returns a list of instrument reference IDs.
#     """

#     instruments = InstrumentData(nubra_sdk)

#     ref_ids = []

#     for name in nubra_names:
#         instrument = instruments.get_instrument_by_nubra_name(name)

#         # SDK returns dict, not object
#         if instrument is None:
#             continue

#         ref_id = instrument.get("ref_id")
#         if ref_id is not None:
#             ref_ids.append(ref_id)

#     return ref_ids


In [10]:
def generate_nifty_option_nubra_names():
    """
    Generates Nubra instrument names for all NIFTY options
    (CE + PE) with expiry 06 Jan 2026 and strikes from
    2,050,000 to 3,145,000 in steps of 5,000.
    
    Returns
    -------
    list[str]
    """

    expiry = "20260106"
    start_strike = 2050000
    end_strike = 3145000
    step = 5000

    nubra_names = []

    strike = start_strike
    while strike <= end_strike:
        nubra_names.append(f"OPT_NIFTY_{expiry}_CE_{strike}")
        nubra_names.append(f"OPT_NIFTY_{expiry}_PE_{strike}")
        strike += step

    return nubra_names

In [12]:
# nubra_names = [
#     "OPT_NIFTY_20260106_PE_2050000",
#     "OPT_NIFTY_20260106_PE_2055000",
#     "OPT_NIFTY_20260106_PE_2060000",
#     # ...
#     "OPT_NIFTY_20260106_PE_3145000",
# ]


nubra_names = generate_nifty_option_nubra_names()


In [14]:
# nubra_names

In [16]:
ref_ids = fetch_instrument_ref_ids(nubra, nubra_names)


In [18]:
ref_ids

[1115157,
 1115158,
 1115159,
 1115160,
 1114439,
 1114440,
 1114441,
 1114442,
 1113746,
 1113747,
 1107728,
 1107729,
 1106789,
 1106790,
 1106791,
 1106792,
 1106793,
 1106794,
 1106795,
 1106796,
 1106797,
 1106798,
 1106799,
 1106800,
 1106801,
 1106802,
 1106803,
 1106804,
 1106805,
 1106806,
 1106807,
 1106808,
 1106809,
 1106810,
 1106811,
 1106812,
 1106813,
 1106814,
 1106815,
 1106816,
 1106817,
 1106818,
 1106819,
 1106820,
 1106821,
 1106822,
 1106823,
 1106824,
 1106825,
 1106826,
 1106827,
 1106828,
 1106829,
 1106830,
 1106831,
 1106832,
 1106833,
 1106834,
 1106835,
 1106836,
 1106837,
 1106838,
 1106839,
 1106840,
 1106841,
 1106842,
 1106843,
 1106844,
 1106845,
 1106846,
 1106847,
 1106848,
 1106849,
 1106850,
 1106851,
 1106852,
 1106853,
 1106854,
 1106855,
 1106856,
 1106857,
 1106858,
 1106859,
 1106860,
 1106861,
 1106862,
 1106863,
 1106864,
 1106865,
 1106866,
 1106867,
 1106868,
 1106869,
 1106870,
 1106871,
 1106872,
 1106873,
 1106874,
 1106875,
 1106876,


## Web Socket threading, connection, subscribing, extracting, auto-close

In [25]:
from nubra_python_sdk.ticker import websocketdata

from nubra_python_sdk.ticker import websocketdata

def start_orderbook_websocket(nubra, ref_ids):
    global ws_thread, market_socket

    ws_stop_event.clear()

    def on_orderbook_data(msg):
        try:
            ref_id = msg.ref_id
            timestamp = msg.timestamp

            valid_bids = [
                o for o in msg.bids
                if o.price is not None and o.quantity is not None
            ]
            valid_asks = [
                o for o in msg.asks
                if o.price is not None and o.quantity is not None
            ]

            if not valid_bids or not valid_asks:
                return

            best_bid = max(valid_bids, key=lambda x: x.price)
            best_ask = min(valid_asks, key=lambda x: x.price)

            if best_bid.price >= best_ask.price:
                return

            snapshot = {
                "timestamp": timestamp,
                "best_bid": best_bid.price,
                "best_bid_qty": best_bid.quantity,
                "best_ask": best_ask.price,
                "best_ask_qty": best_ask.quantity,
            }

            with orderbook_lock:
                latest_orderbooks[ref_id] = snapshot

        except Exception as e:
            print("Orderbook parse error:", repr(e))

    def on_connect(msg):
        print("[WS CONNECTED]", msg)

    def on_close(reason):
        print("[WS CLOSED]", reason)

    def on_error(err):
        print("[WS ERROR]", err)

    market_socket = websocketdata.NubraDataSocket(
        client=nubra,
        on_orderbook_data=on_orderbook_data,
        on_connect=on_connect,
        on_close=on_close,
        on_error=on_error,
    )

    def run():
        market_socket.connect()

        for ref_id in ref_ids:
            market_socket.subscribe([str(ref_id)], data_type="orderbook")

        # 🔴 Controlled loop instead of blocking forever
        while not ws_stop_event.is_set():
            time.sleep(0.2)

        # Graceful shutdown
        try:
            market_socket.close()
        except Exception:
            pass

        print("[WS THREAD EXITED]")

    ws_thread = threading.Thread(target=run, daemon=True)
    ws_thread.start()

    return market_socket


In [31]:
import threading
import time
from collections import defaultdict

latest_orderbooks = {}
orderbook_lock = threading.Lock()

def main():

    # --------------------
    # 1. Authenticate
    # --------------------
    nubra = authenticate_nubra(NubraEnv.PROD)

    # --------------------
    # 2. Instrument universe
    # --------------------
    nubra_names = generate_nifty_option_nubra_names()
    ref_ids = fetch_instrument_ref_ids(nubra, nubra_names)

    print("Subscribed ref_ids:", ref_ids[:5], "... total =", len(ref_ids))

    # --------------------
    # 3. Start WebSocket (FIXED)
    # --------------------
    print("Starting orderbook websocket...")
    market_socket = start_orderbook_websocket(nubra, ref_ids)

    # --------------------
    # 4. Wait for first data
    # --------------------
    print("Waiting for first snapshots...")
    while True:
        with orderbook_lock:
            if len(latest_orderbooks) > 0:
                break
        time.sleep(0.1)

    print("WebSocket live. Snapshots received:",
          len(latest_orderbooks))

    # 🔴 STOP HERE FOR NOW
    # Next steps: warmup → calibration → trading loop


In [33]:
main()


Subscribed ref_ids: [1115157, 1115158, 1115159, 1115160, 1114439] ... total = 440
Starting orderbook websocket...
Waiting for first snapshots...
[WS CONNECTED] WebSocket Connected!
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'int'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'int'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not suppor

In [37]:
time.sleep(2)
with orderbook_lock:
    list(latest_orderbooks.items())[:1]


Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'int' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'int'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'int'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported between instances of 'NoneType' and 'NoneType'
Orderbook parse error: '<' not supported betwee

In [27]:
# data/websocket_client.py

import threading
import time
from datetime import datetime

from nubra_python_sdk.ticker import websocketdata


# ==============================
# Shared state (read by strategy)
# ==============================

# ref_id -> {
#   "timestamp": int,
#   "best_bid": int,
#   "best_ask": int,
#   "bid_qty": int,
#   "ask_qty": int,
#   "bids_5": [(price, qty), ...],
#   "asks_5": [(price, qty), ...],
# }
latest_orderbooks = {}


# ==============================
# WebSocket Callbacks
# ==============================

def on_orderbook_data(msg):
    """
    Called whenever orderbook data is received.
    Extracts:
    - Best bid / ask
    - 5-level bid & ask depth (price, quantity, num_orders)
    Stores latest snapshot per ref_id.
    """

    ref_id = msg.ref_id
    timestamp = msg.timestamp

    bids = msg.bids
    asks = msg.asks

    if not bids or not asks:
        return

    # Sort bids: highest price first
    bids_sorted = sorted(bids, key=lambda x: x.price, reverse=True)

    # Sort asks: lowest price first
    asks_sorted = sorted(asks, key=lambda x: x.price)

    # Extract top 5 levels
    bids_5 = [
        (b.price, b.quantity, b.num_orders)
        for b in bids_sorted[:5]
    ]

    asks_5 = [
        (a.price, a.quantity, a.num_orders)
        for a in asks_sorted[:5]
    ]

    best_bid_price, best_bid_qty, best_bid_orders = bids_5[0]
    best_ask_price, best_ask_qty, best_ask_orders = asks_5[0]

    # Update shared state
    latest_orderbooks[ref_id] = {
        "timestamp": timestamp,
        "best_bid": best_bid_price,
        "best_ask": best_ask_price,
        "bid_qty": best_bid_qty,
        "ask_qty": best_ask_qty,
        "bid_num_orders": best_bid_orders,
        "ask_num_orders": best_ask_orders,
        "bids_5": bids_5,
        "asks_5": asks_5,
    }

    # Strategy hook (to be added later)
    # strategy_on_tick(ref_id, latest_orderbooks[ref_id])



def on_connect(msg):
    pass


def on_close(reason):
    pass


def on_error(err):
    pass


# ==============================
# WebSocket Runner
# ==============================

def start_websocket(nubra_sdk, ref_ids):
    """
    Starts WebSocket in a separate thread and subscribes to orderbook
    data for given ref_ids.
    """

    socket = websocketdata.NubraDataSocket(
        client=nubra_sdk,
        on_orderbook_data=on_orderbook_data,
        on_connect=on_connect,
        on_close=on_close,
        on_error=on_error,
    )

    def run():
        socket.connect()
        socket.subscribe(["1107023"], data_type="orderbook")
        socket.keep_running()

    ws_thread = threading.Thread(target=run)
    ws_thread.daemon = True
    ws_thread.start()




# ==============================
# Session Control (3:30 PM exit)
# ==============================

def stop_websocket_at_330(socket):
    """
    Blocks until 3:30 PM and then closes the WebSocket cleanly.
    Intended to be run in main thread.
    """

    while True:
        now = datetime.now()
        if now.hour > 15 or (now.hour == 15 and now.minute >= 30):
            socket.close()
            break
        time.sleep(1)





# #USE CASE IN MAIN-
# from auth import authenticate_nubra
# from data.websocket_client import (
#     generate_nifty_option_nubra_names,
#     fetch_instrument_ref_ids,
#     start_websocket,
#     stop_websocket_at_330
# )

# # 1. Authenticate
# nubra = authenticate_nubra()

# # 2. Generate instruments
# nubra_names = generate_nifty_option_nubra_names()

# # 3. Fetch ref_ids
# ref_ids = fetch_instrument_ref_ids(nubra, nubra_names)

# # 4. Start WebSocket
# socket = start_websocket(nubra, ref_ids)

# # 5. Run until 3:30 PM
# stop_websocket_at_330(socket)


[Orderbook]  OrderBookWrapper(ref_id=1107019, timestamp=1767067525940659293, last_traded_price=620, last_traded_quantity=65, volume=362310, bids=[Orders(price=620, quantity=1495, num_orders=8), Orders(price=615, quantity=6240, num_orders=17), Orders(price=610, quantity=5525, num_orders=11), Orders(price=605, quantity=3510, num_orders=8), Orders(price=600, quantity=2600, num_orders=6), Orders(price=595, quantity=975, num_orders=4), Orders(price=590, quantity=1235, num_orders=4), Orders(price=585, quantity=650, num_orders=2), Orders(price=580, quantity=195, num_orders=1), Orders(price=575, quantity=195, num_orders=1), Orders(price=570, quantity=650, num_orders=2), Orders(price=560, quantity=1560, num_orders=1), Orders(price=555, quantity=1755, num_orders=1), Orders(price=550, quantity=1755, num_orders=1), Orders(price=545, quantity=195, num_orders=1), Orders(price=540, quantity=1040, num_orders=1), Orders(price=530, quantity=3055, num_orders=2), Orders(price=510, quantity=65, num_orders=

In [31]:
# # 4. Start WebSocket
socket = start_websocket(nubra, ref_ids)


In [21]:
from nubra_python_sdk.ticker import websocketdata
from nubra_python_sdk.start_sdk import InitNubraSdk, NubraEnv


# Initialize the Nubra SDK client
nubra = InitNubraSdk(NubraEnv.PROD)

# Define callback functions

def on_orderbook_data(msg):
    print("[Orderbook] ", msg)

def on_connect(msg):
    print("[status]", msg)

def on_close(reason):
    print(f"Closed: {reason}")

def on_error(err):
    print(f"Error: {err}")

# Initialize WebSocket
socket = websocketdata.NubraDataSocket(
    client=nubra,
    on_orderbook_data=on_orderbook_data,
    on_connect=on_connect,
    on_close=on_close,
    on_error=on_error
    )

socket.connect()
#Subscribe to different data types 
socket.subscribe(["1107010"], data_type="orderbook")

#Infinite loop on the main thread. Nothing after this will run.
#You have to use the pre-defined callbacks to manage subscriptions.
socket.keep_running()

[status] WebSocket Connected!
[Orderbook]  OrderBookWrapper(ref_id=1107010, timestamp=1767068178879583098, last_traded_price=34285, last_traded_quantity=65, volume=205660, bids=[Orders(price=34320, quantity=195, num_orders=1), Orders(price=34315, quantity=130, num_orders=1), Orders(price=34310, quantity=65, num_orders=1), Orders(price=34305, quantity=325, num_orders=3), Orders(price=34300, quantity=325, num_orders=3), Orders(price=34295, quantity=195, num_orders=1), Orders(price=34290, quantity=65, num_orders=1), Orders(price=34285, quantity=65, num_orders=1), Orders(price=34280, quantity=130, num_orders=1), Orders(price=34275, quantity=195, num_orders=2), Orders(price=34265, quantity=130, num_orders=1), Orders(price=34260, quantity=195, num_orders=2), Orders(price=34255, quantity=130, num_orders=1), Orders(price=34250, quantity=195, num_orders=1), Orders(price=34230, quantity=65, num_orders=1), Orders(price=34225, quantity=65, num_orders=1), Orders(price=34220, quantity=130, num_order

KeyboardInterrupt: 

[Orderbook]  OrderBookWrapper(ref_id=1107010, timestamp=1767068333961567100, last_traded_price=36100, last_traded_quantity=65, volume=213005, bids=[Orders(price=35980, quantity=260, num_orders=3), Orders(price=35975, quantity=130, num_orders=1), Orders(price=35970, quantity=325, num_orders=4), Orders(price=35965, quantity=260, num_orders=3), Orders(price=35960, quantity=130, num_orders=1), Orders(price=35955, quantity=65, num_orders=1), Orders(price=35950, quantity=260, num_orders=2), Orders(price=35940, quantity=65, num_orders=1), Orders(price=35930, quantity=130, num_orders=2), Orders(price=35920, quantity=130, num_orders=1), Orders(price=35915, quantity=130, num_orders=1), Orders(price=35910, quantity=130, num_orders=1), Orders(price=35900, quantity=325, num_orders=2), Orders(price=35890, quantity=195, num_orders=2), Orders(price=35860, quantity=130, num_orders=1), Orders(price=35850, quantity=130, num_orders=1), Orders(price=35840, quantity=130, num_orders=1), Orders(price=35835, q

In [23]:
socket.close()

[Orderbook]  OrderBookWrapper(ref_id=1107010, timestamp=1767068336928896680, last_traded_price=36120, last_traded_quantity=65, volume=213395, bids=[Orders(price=35930, quantity=130, num_orders=2), Orders(price=35920, quantity=65, num_orders=1), Orders(price=35915, quantity=260, num_orders=4), Orders(price=35910, quantity=130, num_orders=2), Orders(price=35900, quantity=195, num_orders=1), Orders(price=35895, quantity=65, num_orders=1), Orders(price=35880, quantity=65, num_orders=1), Orders(price=35870, quantity=65, num_orders=1), Orders(price=35865, quantity=195, num_orders=2), Orders(price=35860, quantity=130, num_orders=1), Orders(price=35855, quantity=65, num_orders=1), Orders(price=35850, quantity=390, num_orders=2), Orders(price=35845, quantity=65, num_orders=1), Orders(price=35840, quantity=130, num_orders=1), Orders(price=35830, quantity=130, num_orders=1), Orders(price=35825, quantity=130, num_orders=1), Orders(price=35790, quantity=130, num_orders=1), Orders(price=35785, quant

## mt, v_at, v_bt, bt, at, I_t

In [18]:
# data/data_handler.py

def compute_l1_features(orderbook_snapshot):
    """
    Computes L1 features from the latest orderbook snapshot.

    Parameters
    ----------
    orderbook_snapshot : dict
        One entry from latest_orderbooks[ref_id]

    Returns
    -------
    dict
        {
            "bt": best_bid_price,
            "at": best_ask_price,
            "mt": mid_price,
            "V_bt": volume_at_best_bid,
            "V_at": volume_at_best_ask,
            "I_t": imbalance
        }
    """

    bt = orderbook_snapshot["best_bid"]
    at = orderbook_snapshot["best_ask"]

    V_bt = orderbook_snapshot["bid_qty"]
    V_at = orderbook_snapshot["ask_qty"]

    # Mid price (PDF definition)
    mt = (bt + at) / 2.0

    # Imbalance (PDF definition)
    denom = V_bt + V_at
    if denom > 0:
        I_t = (V_bt - V_at) / denom
    else:
        I_t = 0.0

    return {
        "bt": bt,
        "at": at,
        "mt": mt,
        "V_bt": V_bt,
        "V_at": V_at,
        "I_t": I_t,
    }


In [20]:
# USE CASE IN MAIN-

# snapshot = latest_orderbooks[ref_id]
# l1 = compute_l1_features(snapshot)

# mt = l1["mt"]
# I_t = l1["I_t"]


## EXTRAS- 5lvl metrics for risk control

In [23]:
# data/depth_features.py

# Minimal rolling state for instability (per ref_id)
_prev_depth_state = {}


def compute_depth_risk_features(ref_id, orderbook_snapshot):
    """
    Computes risk-only metrics from 5-level depth.
    No prediction. No trading decisions.

    Parameters
    ----------
    ref_id : int
        Instrument reference ID
    orderbook_snapshot : dict
        One entry from latest_orderbooks[ref_id]

    Returns
    -------
    dict
        {
            "Vbid_5": cumulative_bid_volume_5,
            "Vask_5": cumulative_ask_volume_5,
            "I_5": depth_imbalance_5,
            "liq_conf_bid": liquidity_confidence_bid,
            "liq_conf_ask": liquidity_confidence_ask,
            "instability": instability_score
        }
    """

    bids_5 = orderbook_snapshot["bids_5"]
    asks_5 = orderbook_snapshot["asks_5"]

    # Cumulative volume (5 levels)
    Vbid_5 = 0
    Vask_5 = 0

    # Liquidity confidence proxies
    # (average size per order across 5 levels)
    sum_bid_qty = 0
    sum_bid_orders = 0
    sum_ask_qty = 0
    sum_ask_orders = 0

    for price, qty, n_orders in bids_5:
        Vbid_5 += qty
        sum_bid_qty += qty
        sum_bid_orders += n_orders

    for price, qty, n_orders in asks_5:
        Vask_5 += qty
        sum_ask_qty += qty
        sum_ask_orders += n_orders

    # Depth imbalance (5-level)
    denom = Vbid_5 + Vask_5
    if denom > 0:
        I_5 = (Vbid_5 - Vask_5) / denom
    else:
        I_5 = 0.0

    # Liquidity confidence (risk-only diagnostic)
    # Higher value => fewer, larger orders (less fragmented)
    liq_conf_bid = (
        sum_bid_qty / sum_bid_orders if sum_bid_orders > 0 else 0.0
    )
    liq_conf_ask = (
        sum_ask_qty / sum_ask_orders if sum_ask_orders > 0 else 0.0
    )

    # Instability detection (simple churn metric)
    # Compare current cumulative depth to previous snapshot
    prev = _prev_depth_state.get(ref_id)

    if prev is None:
        instability = 0.0
    else:
        instability = abs(Vbid_5 - prev["Vbid_5"]) + abs(Vask_5 - prev["Vask_5"])

    # Update rolling state (minimal)
    _prev_depth_state[ref_id] = {
        "Vbid_5": Vbid_5,
        "Vask_5": Vask_5,
    }

    return {
        "Vbid_5": Vbid_5,
        "Vask_5": Vask_5,
        "I_5": I_5,
        "liq_conf_bid": liq_conf_bid,
        "liq_conf_ask": liq_conf_ask,
        "instability": instability,
    }


In [25]:
# # USE CASE IN MAIN-
# depth_risk = compute_depth_risk_features(ref_id, snapshot)

# if depth_risk["instability"] > threshold:
#     effective_k *= 1.2


## SIMULATOR L1 (RULES ACCORDING TO PDF)

In [27]:
# execution/simulator.py

def simulate_l1_fills(
    *,
    quoted_bid,
    quoted_ask,
    market_bid,
    market_ask,
    mt,
    Q,
    C,
):
    """
    Deterministic L1 fill simulation (PDF-compliant).

    Rules:
    - If quoted_bid >= market_ask -> BUY 1 unit at market_ask
    - If quoted_ask <= market_bid -> SELL 1 unit at market_bid

    State update:
    - Buy  -> Q += 1, C -= market_ask
    - Sell -> Q -= 1, C += market_bid

    Mark-to-market PnL:
    - PnL = C + Q * mt

    Parameters
    ----------
    quoted_bid : float
        Strategy quoted bid (b̂)
    quoted_ask : float
        Strategy quoted ask (â)
    market_bid : int
        Current best bid (bt)
    market_ask : int
        Current best ask (at)
    mt : float
        Mid price
    Q : int
        Current inventory
    C : float
        Current cash

    Returns
    -------
    tuple
        (Q_new, C_new, pnl, event)

        event: dict suitable for CSV logging
    """

    fill_side = None
    fill_price = None

    # BUY condition
    if quoted_bid >= market_ask:
        Q += 1
        C -= market_ask
        fill_side = "BUY"
        fill_price = market_ask

    # SELL condition
    elif quoted_ask <= market_bid:
        Q -= 1
        C += market_bid
        fill_side = "SELL"
        fill_price = market_bid

    # Mark-to-market PnL
    pnl = C + Q * mt

    # Logger-friendly event (append-only later)
    event = {
        "fill": 1 if fill_side else 0,
        "side": fill_side,          # "BUY", "SELL", or None
        "fill_price": fill_price,  # int or None
        "Q": Q,
        "C": C,
        "mt": mt,
        "pnl": pnl,
        "quoted_bid": quoted_bid,
        "quoted_ask": quoted_ask,
        "market_bid": market_bid,
        "market_ask": market_ask,
    }

    return Q, C, pnl, event


In [29]:
# # USE CASE IN MAIN
# Q, C, pnl, event = simulate_l1_fills(...)

# # later:
# logger.append(timestamp, ref_id, event)


## SIMULATOR WITH TRADING API

In [33]:
# execution/simulator.py

from nubra_python_sdk.trading.trading_data import NubraTrader
from nubra_python_sdk.trading.trading_enum import (
    OrderSideEnum,
    OrderTypeEnumV2,
    ValidityTypeEnumV2,
    PriceTypeEnumV2,
    DeliveryTypeEnum,
    ExchangeEnum,
)


class ExecutionSimulator:
    """
    Execution manager that:
    - Checks PDF fill conditions
    - Places orders via Nubra UAT
    - Tracks pending orders
    """

    def __init__(self, nubra_client):
        self.trader = NubraTrader(nubra_client, version="V2")

        # order_id -> metadata (used when execution arrives)
        self.pending_orders = {}

    def try_execute(
        self,
        *,
        ref_id,
        quoted_bid,
        quoted_ask,
        market_bid,
        market_ask,
        strategy_name,
        tag_prefix,
        lot_size,
    ):
        """
        Checks PDF fill rules and places order if condition is satisfied.

        Returns
        -------
        dict or None
            Order metadata if order placed, else None
        """

        # BUY condition
        if quoted_bid >= market_ask:
            side = OrderSideEnum.ORDER_SIDE_BUY
            price = market_ask

        # SELL condition
        elif quoted_ask <= market_bid:
            side = OrderSideEnum.ORDER_SIDE_SELL
            price = market_bid

        else:
            return None  # no trade signal

        # Place LIMIT order (1 lot)
        response = self.trader.create_order({
            "ref_id": ref_id,
            "order_type": OrderTypeEnumV2.ORDER_TYPE_REGULAR,
            "order_qty": lot_size,
            "order_side": side,
            "order_delivery_type": DeliveryTypeEnum.ORDER_DELIVERY_TYPE_IDAY,
            "validity_type": ValidityTypeEnumV2.DAY,
            "price_type": PriceTypeEnumV2.LIMIT,
            "order_price": int(price),
            "exchange": ExchangeEnum.NSE,
            "tag": f"{tag_prefix}_{strategy_name}",
        })

        order_id = response.order_id

        # Track pending order
        self.pending_orders[order_id] = {
            "ref_id": ref_id,
            "side": side,
            "price": price,
            "strategy": strategy_name,
        }

        return {
            "order_id": order_id,
            "side": side,
            "price": price,
            "strategy": strategy_name,
        }

    def pop_pending(self, order_id):
        """
        Remove and return pending order metadata
        """
        return self.pending_orders.pop(order_id, None)


In [34]:
# #USE CASE IN MAIN-
# def on_trade_update(msg):
#     order_id = msg.order_id
#     filled_qty = msg.trade_qty
#     avg_price = msg.trade_price

#     meta = simulator.pop_pending(order_id)
#     if meta is None:
#         return

#     if meta["side"] == ORDER_SIDE_BUY:
#         Q += filled_qty / lot_size
#         C -= filled_qty * avg_price
#     else:
#         Q -= filled_qty / lot_size
#         C += filled_qty * avg_price

#     pnl = C + Q * mt

#     logger.log_trade(...)


In [36]:
# execution/order_update_client.py

import threading

from nubra_python_sdk.ticker import orderupdate
from nubra_python_sdk.start_sdk import InitNubraSdk, NubraEnv
from nubra_python_sdk.trading.trading_enum import OrderSideEnum, OrderStatusEnum


class OrderUpdateClient:
    """
    Handles real-time order and trade updates from Nubra UAT.
    Updates inventory and logs ONLY executed trades.
    """

    def __init__(
        self,
        nubra_client,
        execution_simulator,
        inventory_manager,
        logger,
    ):
        self.simulator = execution_simulator
        self.inventory_manager = inventory_manager
        self.logger = logger

        self.socket = orderupdate.OrderUpdate(
            client=nubra_client,
            on_order_update=self.on_order_update,
            on_trade_update=self.on_trade_update,
            on_connect=self.on_connect,
            on_close=self.on_close,
            on_error=self.on_error,
        )

    # ---------- WebSocket lifecycle ----------

    def start(self):
        """
        Start OrderUpdate WebSocket in a background thread.
        """

        def run():
            self.socket.connect()
            self.socket.keep_running()

        t = threading.Thread(target=run)
        t.daemon = True
        t.start()

    def close(self):
        self.socket.close()

    # ---------- Callbacks ----------

    def on_connect(self, msg):
        # Connection status only
        pass

    def on_close(self, reason):
        pass

    def on_error(self, err):
        # You may want to log this
        pass

    # ---------- Order updates (status-level) ----------

    def on_order_update(self, msg):
        """
        Receives order status updates.
        Useful for debugging, rejects, cancels.
        """

        # Example usage (optional):
        # if msg.order_status == OrderStatusEnum.ORDER_STATUS_REJECTED:
        #     handle rejection
        pass

    # ---------- Trade updates (execution-level) ----------

    def on_trade_update(self, msg):
        """
        Receives actual trade execution updates.
        THIS is where inventory & cash are updated.
        """

        order_id = msg.order_id
        trade_qty = msg.trade_qty
        trade_price = msg.trade_price
        side = msg.side
        timestamp = msg.last_modified

        if order_id is None or trade_qty is None or trade_price is None:
            return

        # Retrieve metadata from simulator
        meta = self.simulator.pop_pending(order_id)
        if meta is None:
            return  # not our order

        # Update inventory & cash
        self.inventory_manager.apply_trade(
            side=side,
            qty=trade_qty,
            price=trade_price,
        )

        # Compute mark-to-market PnL
        pnl = self.inventory_manager.mark_to_market()

        # Log executed trade
        self.logger.log_trade({
            "timestamp": timestamp,
            "ref_id": meta["ref_id"],
            "strategy": meta["strategy"],
            "side": side,
            "trade_qty": trade_qty,
            "trade_price": trade_price,
            "Q": self.inventory_manager.Q,
            "C": self.inventory_manager.C,
            "pnl": pnl,
            "order_id": order_id,
        })


In [37]:
# #USE CASE IN MAIN
# simulator = ExecutionSimulator(nubra)
# inventory_manager = InventoryManager()
# logger = TradeLogger()

# order_update_client = OrderUpdateClient(
#     nubra,
#     simulator,
#     inventory_manager,
#     logger,
# )

# order_update_client.start()


## BASELINE STRATEGY

In [44]:
# strategy/baseline_strategy.py

def compute_baseline_quotes(mt, tick_size):
    """
    Computes baseline bid and ask quotes.

    Baseline strategy:
    - Static spread
    - No imbalance
    - No inventory skew

    Parameters
    ----------
    mt : float
        Mid price
    tick_size : int
        Minimum price movement (paise)

    Returns
    -------
    tuple
        (b_hat, a_hat)
    """

    # Static spread (PDF definition)
    s0 = 2 * tick_size

    # Baseline quotes
    b_hat = mt - (s0 / 2.0)
    a_hat = mt + (s0 / 2.0)

    return b_hat, a_hat


In [45]:
# # USE CASE IN MAIN-
# b_hat, a_hat = compute_baseline_quotes(mt, tick_size)

# Q, C, pnl, event = simulate_l1_fills(
#     quoted_bid=b_hat,
#     quoted_ask=a_hat,
#     market_bid=bt,
#     market_ask=at,
#     mt=mt,
#     Q=baseline_Q,
#     C=baseline_C,
# )


## STRATEGY ADAPTIVE

In [47]:
# strategy/adaptive_strategy.py

def compute_adaptive_quotes(
    mt,
    I_t,
    Q,
    tick_size,
    alpha,
    k,
    Qmax,
):
    """
    Computes adaptive bid and ask quotes using:
    - Order-book imbalance
    - Inventory skew
    - Fixed tick-based base spread

    Parameters
    ----------
    mt : float
        Mid price
    I_t : float
        L1 imbalance in [-1, 1]
    Q : int
        Current inventory (lots)
    tick_size : int
        Minimum price movement (paise)
    alpha : float
        Imbalance sensitivity parameter
    k : float
        Inventory skew parameter
    Qmax : int
        Maximum allowed inventory (lots)

    Returns
    -------
    tuple
        (b_hat, a_hat)
    """

    # Enforce inventory cap
    if Q > Qmax:
        Q = Qmax
    elif Q < -Qmax:
        Q = -Qmax

    # Base spread (PDF definition)
    s0 = 2 * tick_size

    # Adaptive spread
    st = s0 * (1.0 + alpha * abs(I_t))

    # Adaptive quotes with inventory skew
    b_hat = mt - (st / 2.0) - k * Q
    a_hat = mt + (st / 2.0) - k * Q

    return b_hat, a_hat


In [49]:
# # USE CASE IN MAIN
# while market_open:

#     for each ref_id with new data:

#         snapshot = latest_orderbooks[ref_id]

#         # 1. L1 features
#         l1 = compute_l1_features(snapshot)

#         # 2. Adaptive quotes
#         b_hat, a_hat = compute_adaptive_quotes(
#             mt=l1["mt"],
#             I_t=l1["I_t"],
#             Q=adaptive_Q,
#             tick_size=5,
#             alpha=alpha,
#             k=k,
#             Qmax=Qmax
#         )

#         # 3. Simulate fills
#         adaptive_Q, adaptive_C, adaptive_pnl, event = simulate_l1_fills(
#             quoted_bid=b_hat,
#             quoted_ask=a_hat,
#             market_bid=l1["bt"],
#             market_ask=l1["at"],
#             mt=l1["mt"],
#             Q=adaptive_Q,
#             C=adaptive_C
#         )

#         # 4. Log
#         logger.log(event)


## ALPHA AND K SELECTION

In [53]:
# calibration/warmup_calibration.py

import math

# from strategy.adaptive_strategy import compute_adaptive_quotes
# from execution.simulator import simulate_l1_fills


def _compute_std(values):
    n = len(values)
    if n == 0:
        return 0.0
    mean = sum(values) / n
    var = sum((x - mean) ** 2 for x in values) / n
    return math.sqrt(var)


def _max_drawdown(pnl_series):
    """
    Computes max drawdown from a PnL time series.
    """
    peak = pnl_series[0]
    max_dd = 0.0

    for pnl in pnl_series:
        if pnl > peak:
            peak = pnl
        drawdown = peak - pnl
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd


def _pnl_slope(pnl_series):
    """
    Simple slope proxy:
    (final PnL - initial PnL) / length
    """
    if len(pnl_series) < 2:
        return 0.0
    return (pnl_series[-1] - pnl_series[0]) / len(pnl_series)


def warmup_calibration(
    snapshots,
    *,
    tick_size,
    Qmax,
    alpha_grid,
    k_grid,
    max_std_Q,
    max_drawdown_allowed,
):
    """
    Runs warm-up calibration over first N minutes of data.

    Parameters
    ----------
    snapshots : list[dict]
        Each dict must contain:
        {
            "bt": best_bid,
            "at": best_ask,
            "mt": mid_price,
            "I_t": imbalance
        }

    tick_size : int
        Tick size (paise)

    Qmax : int
        Inventory cap (lots)

    alpha_grid : list[float]
        Candidate alpha values

    k_grid : list[float]
        Candidate k values

    max_std_Q : float
        Maximum allowed std(Q)

    max_drawdown_allowed : float
        Maximum allowed drawdown

    Returns
    -------
    tuple
        (alpha, k)
    """

    viable_candidates = []

    for alpha in alpha_grid:
        for k in k_grid:

            Q = 0
            C = 0.0

            Q_series = []
            pnl_series = []

            for snap in snapshots:
                bt = snap["bt"]
                at = snap["at"]
                mt = snap["mt"]
                I_t = snap["I_t"]

                # Compute adaptive quotes
                b_hat, a_hat = compute_adaptive_quotes(
                    mt=mt,
                    I_t=I_t,
                    Q=Q,
                    tick_size=tick_size,
                    alpha=alpha,
                    k=k,
                    Qmax=Qmax,
                )

                # Simulate fills
                Q, C, pnl, _ = simulate_l1_fills(
                    quoted_bid=b_hat,
                    quoted_ask=a_hat,
                    market_bid=bt,
                    market_ask=at,
                    mt=mt,
                    Q=Q,
                    C=C,
                )

                Q_series.append(Q)
                pnl_series.append(pnl)

            # Risk metrics
            std_Q = _compute_std(Q_series)
            max_dd = _max_drawdown(pnl_series)
            slope = _pnl_slope(pnl_series)

            # Constraint-based filtering
            if (
                std_Q <= max_std_Q
                and max_dd <= max_drawdown_allowed
                and slope > 0.0
            ):
                viable_candidates.append(
                    (alpha, k, std_Q, max_dd, slope)
                )

    # Fallback if nothing passes constraints
    if not viable_candidates:
        # Most conservative default
        return alpha_grid[0], k_grid[0]

    # Choose the most stable candidate:
    # primary: lowest std(Q)
    # secondary: lowest drawdown
    viable_candidates.sort(key=lambda x: (x[2], x[3]))

    best_alpha, best_k, _, _, _ = viable_candidates[0]

    return best_alpha, best_k


In [55]:
# # USE IN MAIN-
# alpha, k = warmup_calibration(
#     snapshots,
#     tick_size=5,
#     Qmax=10,
#     alpha_grid=[0.5, 0.7, 0.9],
#     k_grid=[0.05, 0.1, 0.15],
#     max_std_Q=3.0,
#     max_drawdown_allowed=some_value,
# )


### RISK INVENTORY

In [57]:
# risk/inventory_manager.py

from nubra_python_sdk.trading.trading_enum import OrderSideEnum


class InventoryManager:
    """
    Manages inventory, cash, and mark-to-market PnL.
    Updates state ONLY on confirmed trade executions.
    """

    def __init__(self, *, lot_size, Qmax, initial_cash=0.0):
        """
        Parameters
        ----------
        lot_size : int
            Lot size of the instrument (e.g., 65 for NIFTY options)
        Qmax : int
            Maximum allowed inventory in lots
        initial_cash : float
            Starting cash (paise)
        """

        self.lot_size = lot_size
        self.Qmax = Qmax

        self.Q = 0          # inventory in lots
        self.C = initial_cash  # cash in paise

        self._last_mt = 0.0

    # ---------- Trade application ----------

    def apply_trade(self, *, side, qty, price):
        """
        Apply a confirmed trade execution.

        Parameters
        ----------
        side : OrderSideEnum
            BUY or SELL
        qty : int
            Executed quantity (contracts/shares)
        price : int
            Execution price (paise)
        """

        # Convert quantity to lots
        lots = qty / self.lot_size

        if side == OrderSideEnum.ORDER_SIDE_BUY:
            self.Q += lots
            self.C -= qty * price

        elif side == OrderSideEnum.ORDER_SIDE_SELL:
            self.Q -= lots
            self.C += qty * price

        # Enforce hard inventory cap
        if self.Q > self.Qmax:
            self.Q = self.Qmax
        elif self.Q < -self.Qmax:
            self.Q = -self.Qmax

    # ---------- Mark-to-market ----------

    def update_mid_price(self, mt):
        """
        Update latest mid price for PnL calculation.
        """
        self._last_mt = mt

    def mark_to_market(self):
        """
        Compute mark-to-market PnL.

        Returns
        -------
        float
            Current PnL
        """
        return self.C + self.Q * self._last_mt

    # ---------- State access ----------

    def get_state(self):
        """
        Returns current inventory and cash.
        """
        return {
            "Q": self.Q,
            "C": self.C,
        }


In [58]:
# #USE CASE IN MAIN
# On every market data update (in main loop)
# inventory_manager.update_mid_price(mt)

# On every confirmed trade execution (order-update WS)
# inventory_manager.apply_trade(
#     side=msg.side,
#     qty=msg.trade_qty,
#     price=msg.trade_price,
# )

# pnl = inventory_manager.mark_to_market()

## LOGGER

In [61]:
# utils/logger.py

import csv
import time


class TradeLogger:
    """
    In-memory trade logger.
    Writes CSV once at the end of the session.
    """

    def __init__(self):
        self.rows = []

    def log_trade(self, event):
        """
        Append one executed trade event.

        Expected keys in event dict:
        - timestamp
        - ref_id
        - strategy
        - side
        - trade_qty
        - trade_price
        - Q
        - C
        - pnl
        - order_id

        Optional (if available):
        - quoted_bid
        - quoted_ask
        - market_bid
        - market_ask
        """

        row = {
            "timestamp": event.get("timestamp", int(time.time() * 1e9)),
            "ref_id": event.get("ref_id"),
            "strategy": event.get("strategy"),
            "side": event.get("side"),
            "trade_qty": event.get("trade_qty"),
            "trade_price": event.get("trade_price"),
            "quoted_bid": event.get("quoted_bid"),
            "quoted_ask": event.get("quoted_ask"),
            "market_bid": event.get("market_bid"),
            "market_ask": event.get("market_ask"),
            "Q": event.get("Q"),
            "C": event.get("C"),
            "pnl": event.get("pnl"),
            "order_id": event.get("order_id"),
        }

        self.rows.append(row)

    def write_csv(self, filepath):
        """
        Write all logged trades to CSV.
        Call ONLY after trading ends.
        """

        if not self.rows:
            return

        fieldnames = list(self.rows[0].keys())

        with open(filepath, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.rows)


In [63]:
# # USE CASE IN MAIN
# In order_update_client.py (already done)
# self.logger.log_trade({
#     "timestamp": timestamp,
#     "ref_id": meta["ref_id"],
#     "strategy": meta["strategy"],
#     "side": side,
#     "trade_qty": trade_qty,
#     "trade_price": trade_price,
#     "Q": inventory_manager.Q,
#     "C": inventory_manager.C,
#     "pnl": pnl,
#     "order_id": order_id,
# })

# At the end of main.py
# logger.write_csv("logs/adaptive_trades.csv")

#### MAIN

In [ ]:
# main.py

import time
from datetime import datetime, time as dtime

# --------------------
# AUTH
# --------------------
from auth import authenticate_nubra

# # --------------------
# # MARKET DATA
# # --------------------
# from data.websocket_client import (
#     generate_nifty_option_nubra_names,
#     fetch_instrument_ref_ids,
#     start_websocket,
#     stop_websocket_at_330,
#     latest_orderbooks,
# )

# from data.data_handler import compute_l1_features
# from data.depth_features import compute_depth_risk_features

# # --------------------
# # STRATEGIES
# # --------------------
# from strategy.baseline_strategy import compute_baseline_quotes
# from strategy.adaptive_strategy import compute_adaptive_quotes

# # --------------------
# # EXECUTION
# # --------------------
# from execution.simulator import ExecutionSimulator
# from execution.order_update_client import OrderUpdateClient

# # --------------------
# # RISK & PORTFOLIO
# # --------------------
# from risk.inventory_manager import InventoryManager

# # --------------------
# # CALIBRATION
# # --------------------
# from calibration.warmup_calibration import warmup_calibration

# # --------------------
# # LOGGING
# # --------------------
# from utils.logger import TradeLogger


# ============================================================
# CONFIG (move to config/config.py later)
# ============================================================

TICK_SIZE = 5
LOT_SIZE = 65
QMAX = 10

ALPHA_GRID = [0.5, 0.7, 0.9]
K_GRID = [0.05, 0.1, 0.15]

WARMUP_MINUTES = 5
MAX_STD_Q = 3.0
MAX_DRAWDOWN = 5000  # paise (example)

# ============================================================
# MAIN
# ============================================================

def main():

    # --------------------
    # 1. Authenticate
    # --------------------
    nubra = authenticate_nubra(NubraEnv.PROD)
    nubra = authenticate_nubra(NubraEnv.UAT)

    # --------------------
    # 2. Instrument universe
    # --------------------
    nubra_names = generate_nifty_option_nubra_names()
    ref_ids = fetch_instrument_ref_ids(nubra, nubra_names)
    print(ref_ids)

    # --------------------
    # 3. Start market data WebSocket
    print("starting ws")
    # --------------------
    market_socket = start_websocket(nubra, ref_ids)

    # --------------------
    # 4. Core components
    # --------------------
    simulator = ExecutionSimulator(nubra)

    inventory_baseline = InventoryManager(
        lot_size=LOT_SIZE, Qmax=QMAX
    )
    inventory_adaptive = InventoryManager(
        lot_size=LOT_SIZE, Qmax=QMAX
    )

    logger = TradeLogger()
    
    order_updates_baseline = OrderUpdateClient(
        nubra_client=nubra,
        execution_simulator=simulator,
        inventory_manager=inventory_baseline,
        logger=logger,
    )
    
    order_updates_adaptive = OrderUpdateClient(
        nubra_client=nubra,
        execution_simulator=simulator,
        inventory_manager=inventory_adaptive,
        logger=logger,
    )
    
    order_updates_baseline.start()
    order_updates_adaptive.start()

    # --------------------
    # 5. Warm-up snapshot collection
    # --------------------
    snapshots = []
    warmup_start = time.time()

    while time.time() - warmup_start < WARMUP_MINUTES * 60:
        for ref_id, snapshot in latest_orderbooks.items():

            l1 = compute_l1_features(snapshot)

            snapshots.append({
                "bt": l1["bt"],
                "at": l1["at"],
                "mt": l1["mt"],
                "I_t": l1["I_t"],
            })

        time.sleep(0.01)  # tiny sleep to avoid busy loop

    # --------------------
    # 6. Calibration
    # --------------------
    alpha, k = warmup_calibration(
        snapshots=snapshots,
        tick_size=TICK_SIZE,
        Qmax=QMAX,
        alpha_grid=ALPHA_GRID,
        k_grid=K_GRID,
        max_std_Q=MAX_STD_Q,
        max_drawdown_allowed=MAX_DRAWDOWN,
    )

    # --------------------
    # 7. Live trading loop
    # --------------------
    while datetime.now().time() < dtime(15, 30):
    
        for ref_id, snapshot in latest_orderbooks.items():
    
            # ---------- L1 features ----------
            l1 = compute_l1_features(snapshot)
    
            bt = l1["bt"]
            at = l1["at"]
            mt = l1["mt"]
            I_t = l1["I_t"]
    
            # Update MTM price for both strategies
            inventory_baseline.update_mid_price(mt)
            inventory_adaptive.update_mid_price(mt)
    
            # =====================================================
            # BASELINE STRATEGY
            # =====================================================
            b_hat_base, a_hat_base = compute_baseline_quotes(
                mt=mt,
                tick_size=TICK_SIZE
            )
    
            simulator.try_execute(
                ref_id=ref_id,
                quoted_bid=b_hat_base,
                quoted_ask=a_hat_base,
                market_bid=bt,
                market_ask=at,
                strategy_name="baseline",
                tag_prefix="mm",
                lot_size=LOT_SIZE,
            )
    
            # =====================================================
            # ADAPTIVE STRATEGY
            # =====================================================
            b_hat_adapt, a_hat_adapt = compute_adaptive_quotes(
                mt=mt,
                I_t=I_t,
                Q=inventory_adaptive.Q,
                tick_size=TICK_SIZE,
                alpha=alpha,
                k=k,
                Qmax=QMAX,
            )
    
            simulator.try_execute(
                ref_id=ref_id,
                quoted_bid=b_hat_adapt,
                quoted_ask=a_hat_adapt,
                market_bid=bt,
                market_ask=at,
                strategy_name="adaptive",
                tag_prefix="mm",
                lot_size=LOT_SIZE,
            )
    
        time.sleep(0.005)

    # --------------------
    # 8. Shutdown & logs
    # --------------------
    stop_websocket_at_330(market_socket)
    order_updates.close()

    logger.write_csv("logs/adaptive_trades.csv")


# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    main()


[1115157, 1115158, 1115159, 1115160, 1114439, 1114440, 1114441, 1114442, 1113746, 1113747, 1107728, 1107729, 1106789, 1106790, 1106791, 1106792, 1106793, 1106794, 1106795, 1106796, 1106797, 1106798, 1106799, 1106800, 1106801, 1106802, 1106803, 1106804, 1106805, 1106806, 1106807, 1106808, 1106809, 1106810, 1106811, 1106812, 1106813, 1106814, 1106815, 1106816, 1106817, 1106818, 1106819, 1106820, 1106821, 1106822, 1106823, 1106824, 1106825, 1106826, 1106827, 1106828, 1106829, 1106830, 1106831, 1106832, 1106833, 1106834, 1106835, 1106836, 1106837, 1106838, 1106839, 1106840, 1106841, 1106842, 1106843, 1106844, 1106845, 1106846, 1106847, 1106848, 1106849, 1106850, 1106851, 1106852, 1106853, 1106854, 1106855, 1106856, 1106857, 1106858, 1106859, 1106860, 1106861, 1106862, 1106863, 1106864, 1106865, 1106866, 1106867, 1106868, 1106869, 1106870, 1106871, 1106872, 1106873, 1106874, 1106875, 1106876, 1106877, 1106878, 1106879, 1106880, 1106881, 1106882, 1106883, 1106884, 1106885, 1106886, 1106887, 

🔑 Enter your MPIN:  8080


Deleted: auth_data.db.bak
Deleted: auth_data.db.dat
Deleted: auth_data.db.dir
 Invalid MPIN (2 attempts left). Please try again.
 Invalid MPIN (1 attempts left). Please try again.
 Invalid MPIN (0 attempts left). Please try again.
verify_existing_session failed: MPIN verification failed after multiple attempts.
Invalid session, re-authenticating...


📱 Enter your phone number:  9589265871
🔑 Enter your MPIN:  8080
📱 Enter your phone number:  9589265871


### THE END

In [13]:
# %%writefile data_websocket_client.py

from datetime import datetime, time as dtime
import time

from nubra_python_sdk.ticker import websocketdata
from nubra_python_sdk.marketdata.market_data import MarketData


def fetch_nifty_spot_ltp(nubra_client):
    """
    Fetch NIFTY spot LTP once at startup.

    Logic:
    - If time ∈ [09:15, 15:30]:
        Try WebSocket for max 2 minutes
        If no LTP → fallback to historical
    - Else:
        Use historical directly

    Returns:
        int: NIFTY spot price in paise
    """

    # ---------- Market hours check (IST assumed) ----------
    now = datetime.now().time()
    market_open = dtime(9, 15)
    market_close = dtime(15, 30)

    use_websocket = market_open <= now <= market_close

    # ---------- WebSocket path ----------
    if use_websocket:
        spot_price = {"value": None}

        def on_index_data(msg):
            if msg.indexname == "NIFTY":
                spot_price["value"] = msg.index_value
                socket.close()

        socket = websocketdata.NubraDataSocket(
            client=nubra_client,
            on_index_data=on_index_data,
            on_connect=lambda msg: None,
            on_close=lambda reason: None,
            on_error=lambda err: None
        )

        socket.connect()
        socket.subscribe(
            symbols=["NIFTY"],
            data_type="index",
            exchange="NSE"
        )

        start_ts = time.monotonic()
        timeout_sec = 120.0

        while spot_price["value"] is None:
            socket.keep_running(timeout=1.0)
            if time.monotonic() - start_ts > timeout_sec:
                socket.close()
                break

        if spot_price["value"] is not None:
            return int(spot_price["value"])

    # ---------- Historical fallback (FIXED & API-CORRECT) ----------
    md = MarketData(nubra_client)
    
    result = md.historical_data({
        "exchange": "NSE",
        "type": "INDEX",
        "values": ["NIFTY"],
        "fields": ["close"],
        "startDate": "2000-01-01T00:00:00.000Z",  # required by validator
        "endDate": "2100-01-01T00:00:00.000Z",    # required by validator
        "interval": "30m",
        "intraDay": True,
        "realTime": False
    })
    
    # result is MarketChartsResponse
    for chart_data in result.result:
        for symbol_dict in chart_data.values:
            if "NIFTY" in symbol_dict:
                stock_chart = symbol_dict["NIFTY"]
                close_series = stock_chart.close
                if close_series:
                    return int(close_series[-1].value)
    
    raise RuntimeError("Failed to fetch NIFTY spot price from historical data")



# #USE CASE IN MAIN
# from data_websocket_client import fetch_nifty_spot_ltp
# nifty_spot_ltp = fetch_nifty_spot_ltp(nubra)
# print(nifty_spot_ltp)





# #WEBSOCKET OG-
# from nubra_python_sdk.ticker import websocketdata
# from nubra_python_sdk.start_sdk import NubraEnv


# def fetch_nifty_spot_ltp(nubra_client):
#     """
#     Fetches NIFTY index spot price (LTP) once using WebSocket.
#     Blocks until first valid tick is received, then disconnects.

#     Returns:
#         int: NIFTY index value (paise)
#     """

#     spot_price = {"value": None}

#     def on_index_data(msg):
#         # Capture first index tick and close socket
#         if msg.indexname == "NIFTY":
#             spot_price["value"] = msg.index_value
#             socket.close()

#     def on_connect(msg):
#         pass

#     def on_close(reason):
#         pass

#     def on_error(err):
#         raise RuntimeError(f"Index socket error: {err}")

#     socket = websocketdata.NubraDataSocket(
#         client=nubra_client,
#         on_index_data=on_index_data,
#         on_connect=on_connect,
#         on_close=on_close,
#         on_error=on_error
#     )

#     socket.connect()
#     socket.subscribe(
#         symbols=["NIFTY"],
#         data_type="index",
#         exchange="NSE"
#     )

#     # Block until LTP is received and socket is closed
#     socket.keep_running()

#     return spot_price["value"]



# # from data.websocket_client import fetch_nifty_spot_ltp
# nubra = InitNubraSdk(NubraEnv.PROD)
# nifty_spot = fetch_nifty_spot_ltp(nubra)
# print(nifty_spot)


In [15]:
# from data_websocket_client import fetch_nifty_spot_ltp
nifty_spot_ltp = fetch_nifty_spot_ltp(nubra)
print(nifty_spot_ltp)

2594210


In [39]:
nubra = authenticate_nubra(NubraEnv.UAT)

In [ ]:
from datetime import datetime, time as dtime
import time

from nubra_python_sdk.ticker import websocketdata
from nubra_python_sdk.marketdata.market_data import MarketData


def fetch_nifty_spot_ltp(nubra_client):
    """
    Fetch NIFTY spot LTP once at startup.

    Logic:
    - If time ∈ [09:15, 15:30]:
        Try WebSocket for max 2 minutes
        If no LTP → fallback to historical
    - Else:
        Use historical directly

    Returns:
        int: NIFTY spot price in paise
    """

    # ---------- Market hours check (IST assumed) ----------
    now = datetime.now().time()
    market_open = dtime(9, 15)
    market_close = dtime(15, 30)

    use_websocket = market_open <= now <= market_close

    # ---------- WebSocket path ----------
    if use_websocket:
        spot_price = {"value": None}

        def on_index_data(msg):
            if msg.indexname == "NIFTY":
                spot_price["value"] = msg.index_value
                socket.close()

        socket = websocketdata.NubraDataSocket(
            client=nubra_client,
            on_index_data=on_index_data,
            on_connect=lambda msg: None,
            on_close=lambda reason: None,
            on_error=lambda err: None
        )

        socket.connect()
        socket.subscribe(
            symbols=["NIFTY"],
            data_type="index",
            exchange="NSE"
        )

        start_ts = time.monotonic()
        timeout_sec = 120.0

        while spot_price["value"] is None:
            socket.keep_running(timeout=1.0)
            if time.monotonic() - start_ts > timeout_sec:
                socket.close()
                break

        if spot_price["value"] is not None:
            return int(spot_price["value"])

    # ---------- Historical fallback (FIXED & API-CORRECT) ----------
    md = MarketData(nubra_client)
    
    result = md.historical_data({
        "exchange": "NSE",
        "type": "INDEX",
        "values": ["NIFTY"],
        "fields": ["close"],
        "startDate": "2000-01-01T00:00:00.000Z",  # required by validator
        "endDate": "2100-01-01T00:00:00.000Z",    # required by validator
        "interval": "30m",
        "intraDay": True,
        "realTime": False
    })
    
    # result is MarketChartsResponse
    for chart_data in result.result:
        for symbol_dict in chart_data.values:
            if "NIFTY" in symbol_dict:
                stock_chart = symbol_dict["NIFTY"]
                close_series = stock_chart.close
                if close_series:
                    return int(close_series[-1].value)
    
    raise RuntimeError("Failed to fetch NIFTY spot price from historical data")



# #USE CASE IN MAIN
# from data_websocket_client import fetch_nifty_spot_ltp

nifty_spot_ltp = fetch_nifty_spot_ltp(nubra)
# print(nifty_spot_ltp)


In [ ]:
!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple nubra-sdk

In [17]:
# %%writefile 


In [3]:
# %%writefile auth.py

# !pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple nubra-sdk
from nubra_python_sdk.start_sdk import InitNubraSdk, NubraEnv
from nubra_python_sdk.marketdata.market_data import MarketData
# Initialize SDK
# nubra = InitNubraSdk(NubraEnv.PROD)
# # nubra = InitNubraSdk(NubraEnv.UAT, env_creds=True)
# print("ss")

In [24]:
nubra = InitNubraSdk(NubraEnv.UAT)


In [3]:
# # %%writefile auth.py

# # auth/auth.py
# from nubra_python_sdk.start_sdk import InitNubraSdk, NubraEnv

# def authenticate_nubra():
#     """
#     Performs one-time interactive authentication using PROD environment
#     (phone number, OTP, MPIN), then returns a UAT SDK instance for trading.
#     """

#     # Step 1: Authenticate via PROD (interactive)
#     nubra_prod = InitNubraSdk(NubraEnv.PROD)
#     # - User enters phone number , Receives OTP , Enters MPIN

#     # Step 2: Switch to UAT for market data & simulation
#     nubra_uat = InitNubraSdk(NubraEnv.UAT)

#     return nubra_uat



# #USE IN MAIN FILE
# # from auth.auth import authenticate_nubra
# nubra = authenticate_nubra()

In [2]:
# from nubra_python_sdk.marketdata.market_data import MarketData
# from nubra_python_sdk.refdata.instruments import InstrumentData
# from nubra_python_sdk.start_sdk import InitNubraSdk
# from typing import List, Optional, Dict, Any
# # Initialize instruments master data with the client
# instruments = InstrumentData(nubra)


In [5]:
# data/websocket_client.py

# data/websocket_client.py

from nubra_python_sdk.ticker import websocketdata
from nubra_python_sdk.start_sdk import NubraEnv


def fetch_nifty_spot_ltp(nubra_client):
    """
    Fetches NIFTY index spot price (LTP) once using WebSocket.
    Blocks until first valid tick is received, then disconnects.

    Returns:
        int: NIFTY index value (paise)
    """

    spot_price = {"value": None}

    def on_index_data(msg):
        # Capture first index tick and close socket
        if msg.indexname == "NIFTY":
            spot_price["value"] = msg.index_value
            socket.close()

    def on_connect(msg):
        pass

    def on_close(reason):
        pass

    def on_error(err):
        raise RuntimeError(f"Index socket error: {err}")

    socket = websocketdata.NubraDataSocket(
        client=nubra_client,
        on_index_data=on_index_data,
        on_connect=on_connect,
        on_close=on_close,
        on_error=on_error
    )

    socket.connect()
    socket.subscribe(
        symbols=["NIFTY"],
        data_type="index",
        exchange="NSE"
    )

    # Block until LTP is received and socket is closed
    socket.keep_running()

    return spot_price["value"]



# from data.websocket_client import fetch_nifty_spot_ltp
nubra = InitNubraSdk(NubraEnv.PROD)
nifty_spot = fetch_nifty_spot_ltp(nubra)
print(nifty_spot)

📱 Enter your phone number:  9589265871


🔄 Sending OTP to: 9589265871


🔐 Enter OTP:  167673
🔑 Enter your MPIN:  8080


✅ Login successful.


KeyboardInterrupt: 

In [20]:
from nubra_python_sdk.marketdata.market_data import MarketData
from nubra_python_sdk.start_sdk import InitNubraSdk, NubraEnv

# Initialize the Nubra SDK client
# Use NubraEnv.UAT for testing or NubraEnv.PROD for production
nubra = InitNubraSdk(NubraEnv.PROD)  # or NubraEnv.PROD

##using totp login and .env file 
#nubra = InitNubraSdk(NubraEnv.UAT, totp_login= True ,env_creds = True)

# Initialize MarketData with the client
mdInstance = MarketData(nubra)
result = mdInstance.historical_data({
    "exchange": "NSE",
    "type": "INDEX",
    "values": ["NIFTY"],
    "fields": ["close"],
    "startDate": "",
    "endDate": "",
    "interval": "30m",
    "intraDay": True, # If set to True, then startDate and endDate will be ignored, and the current date will be used.
    "realTime": False
})

result

MarketChartsResponse(market_time='2025-12-29T10:00:00Z', message='charts', result=[ChartData(exchange='NSE', type='INDEX', values=[{'NIFTY': StockChart(open=[], high=[], low=[], close=[TimeSeriesPoint(timestamp=1766979900000000000, value=2608040), TimeSeriesPoint(timestamp=1766981700000000000, value=2603455), TimeSeriesPoint(timestamp=1766983500000000000, value=2600340), TimeSeriesPoint(timestamp=1766985300000000000, value=2598370), TimeSeriesPoint(timestamp=1766987100000000000, value=2597605), TimeSeriesPoint(timestamp=1766988900000000000, value=2597845), TimeSeriesPoint(timestamp=1766990700000000000, value=2596030), TimeSeriesPoint(timestamp=1766992500000000000, value=2595410), TimeSeriesPoint(timestamp=1766994300000000000, value=2593970), TimeSeriesPoint(timestamp=1766996100000000000, value=2594450), TimeSeriesPoint(timestamp=1766997900000000000, value=2593575), TimeSeriesPoint(timestamp=1766999700000000000, value=2594410), TimeSeriesPoint(timestamp=1767001500000000000, value=259421

In [22]:
result.info()

AttributeError: 'MarketChartsResponse' object has no attribute 'info'

In [17]:
# Access values only after checking data exists in any variable as based on request some fields may not have value
print(f"Market Time: {result.market_time}")
print(f"Message: {result.message}")

for data in result.result:
    print(f"Exchange: {data.exchange}")
    print(f"Type: {data.type}")
    for stock_data in data.values:
        for symbol, values in stock_data.items():
            print(f"\n{symbol}:")
            print(f"  Close: {[{'timestamp': v.timestamp, 'value': v.value} for v in values.close]}")
            print(f"  High: {[{'timestamp': v.timestamp, 'value': v.value} for v in values.high]}")
            print(f"  Low: {[{'timestamp': v.timestamp, 'value': v.value} for v in values.low]}")
            print(f"  Open: {[{'timestamp': v.timestamp, 'value': v.value} for v in values.open]}")
            print(f"  Volume: {[{'timestamp': v.timestamp, 'value': v.value} for v in values.cumulative_volume]}")


Market Time: 2025-12-29T10:00:00Z
Message: charts
Exchange: NSE
Type: INDEX

NIFTY:
  Close: []
  High: []
  Low: []
  Open: []
  Volume: []


In [28]:
nubra = InitNubraSdk(NubraEnv.UAT)


In [31]:
from nubra_python_sdk.refdata.instruments import InstrumentData
from nubra_python_sdk.start_sdk import InitNubraSdk
from typing import List, Optional, Dict, Any

class InstrumentFinder:
    """
    A utility class to find instruments using various filters.
    This class can be imported and used in other scripts to find instruments
    based on different criteria like exchange, asset, derivative type, etc.
    """
    def __init__(self):
        self.client = InitNubraSdk(NubraEnv.UAT)
        self.instruments = InstrumentData(self.client)
        self.instruments_df = None
        self._load_reference_data()

    def _load_reference_data(self):
        if self.instruments_df is None:
            try:
                self.instruments_df = self.instruments.get_instruments_dataframe()
                if self.instruments_df is not None:
                    self.instruments_df.set_index(['ref_id'], inplace=True)
            except Exception as e:
                print(f"Error loading reference data: {e}")
                self.instruments_df = None

    def get_instruments(self, 
                       exchange: Optional[str] = None,
                       asset: Optional[str] = None,
                       derivative_type: Optional[str] = None,
                       asset_type: Optional[str] = None,
                       expiry: Optional[str] = None,
                       strike_price: Optional[float] = None,
                       option_type: Optional[str] = None) -> List[Dict[str, Any]]:
        """
        Find instruments based on the provided filters.
        """
        try:
            if self.instruments_df is None:
                self._load_reference_data()
                if self.instruments_df is None:
                    return []
            filtered_df = self.instruments_df.copy()
            if exchange:
                filtered_df = filtered_df[filtered_df['exchange'] == exchange]
            if asset:
                filtered_df = filtered_df[filtered_df['asset'] == asset]
            if derivative_type:
                filtered_df = filtered_df[filtered_df['derivative_type'] == derivative_type]
            if asset_type:
                filtered_df = filtered_df[filtered_df['asset_type'] == asset_type]
            if expiry:
                filtered_df = filtered_df[filtered_df['expiry'] == expiry]
            if strike_price is not None:
                filtered_df = filtered_df[filtered_df['strike_price'] == strike_price]
            if option_type:
                filtered_df = filtered_df[filtered_df['option_type'] == option_type]
            return filtered_df.reset_index().to_dict('records')
        except Exception as e:
            print(f"Error finding instruments: {e}")
            return []

# Example usage
finder = InstrumentFinder()
instruments = finder.get_instruments(
    exchange='NSE',
    asset='NIFTY',
    derivative_type='OPT'
)
print(f"\nFound {len(instruments)} instruments for HDFCBANK stock:")
for instrument in instruments:
    print(f"\nInstrument details:")
    print(instrument)


Error loading reference data: "None of ['ref_id'] are in the columns"
Error loading reference data: "None of ['ref_id'] are in the columns"

Found 0 instruments for HDFCBANK stock:


In [3]:
from nubra_python_sdk.refdata.instruments import InstrumentData
from nubra_python_sdk.start_sdk import InitNubraSdk
from typing import List, Optional, Dict, Any

# Get all instruments as a pandas DataFrame
nubra = InitNubraSdk(NubraEnv.UAT)  
instruments = InstrumentData(nubra)
instruments_df = instruments.get_instruments_dataframe()
print(f"Total instruments: {len(instruments_df)}\n\n")
# instruments_df.to_csv(('uat_sym.csv'))

Total instruments: 91205




In [23]:
nubra = authenticate_nubra(NubraEnv.PROD)

In [25]:
instrument = instruments.get_instrument_by_nubra_name("OPT_NIFTY_20260106_PE_3145000") 
print(f"Instrument details: {instrument}\n\n")


Instrument details: {'msg': 'Check for Nubra name'}




In [17]:
from nubra_python_sdk.start_sdk import InitNubraSdk, NubraEnv
from nubra_python_sdk.trading.trading_data import NubraTrader
from nubra_python_sdk.trading.trading_enum import ( 
    OrderSideEnum, 
    OrderTypeEnumV2,
    ValidityTypeEnumV2,
    PriceTypeEnumV2
)

# Initialize the Nubra SDK client
# Use NubraEnv.UAT for testing or NubraEnv.PROD for production
nubra = InitNubraSdk(NubraEnv.UAT)  # or NubraEnv.PROD

##using totp login and .env file 
#nubra = InitNubraSdk(NubraEnv.UAT, totp_login= True ,env_creds = True)

trade = NubraTrader(nubra, version= "V2")

result = trade.create_order({
    "ref_id": 250486,
    "order_type": "ORDER_TYPE_STOPLOSS", 
    "order_qty": 100,
    "order_side": "ORDER_SIDE_BUY", 
    "order_delivery_type": "ORDER_DELIVERY_TYPE_CNC", 
    "validity_type" : "DAY", 
    "price_type": "LIMIT", 
    "order_price": 11400,
    "exchange": "NSE",
    "tag": "order_test",
    "algo_params": {
    "trigger_price": 11380
 }
})


🔑 Enter your MPIN:  8080


MPIN verification error (2 attempts left): 'NubraTrader' object has no attribute 'API_BASE_URL'


🔑 Enter your MPIN:  8080


MPIN verification error (1 attempts left): 'NubraTrader' object has no attribute 'API_BASE_URL'


🔑 Enter your MPIN:  


MPIN verification error (0 attempts left): 'NubraTrader' object has no attribute 'API_BASE_URL'
verify_existing_session failed: MPIN verification failed after multiple attempts.
Invalid session, re-authenticating...
Login failed: 'NubraTrader' object has no attribute '_InitNubraSdk__phone_number'
Deleted: auth_data.db.bak
Deleted: auth_data.db.dat
Deleted: auth_data.db.dir


UnauthorizedError: Unauthorized even after re-login

In [11]:
# Funds and Margin Details
print(f"Client Code: {result.port_funds_and_margin.client_code}")
print(f"Start of Day Funds: {result.port_funds_and_margin.start_of_day_funds}")
print(f"Cash Blocked for CNC Traded: {result.port_funds_and_margin.cash_blocked_cnc_traded}")
print(f"Cash for CNC Traded & Open Positions: {result.port_funds_and_margin.cash_cnc_traded_and_open}")
print(f"MTM for EQ IDAY + CNC: {result.port_funds_and_margin.mtm_eq_iday_cnc}")
print(f"MTM for EQ Delivery: {result.port_funds_and_margin.mtm_eq_delivery}")
print(f"Net Trading Amount: {result.port_funds_and_margin.net_trading_amount}")
print(f"Net Withdrawal Amount: {result.port_funds_and_margin.net_withdrawal_amount}")
print(f"Start of Day Collateral: {result.port_funds_and_margin.start_of_day_collateral}")
print(f"Total Collateral: {result.port_funds_and_margin.total_collateral}")
print(f"Net Margin Available: {result.port_funds_and_margin.net_margin_available}")


Client Code: I01075
Start of Day Funds: 0
Cash Blocked for CNC Traded: 0
Cash for CNC Traded & Open Positions: 0
MTM for EQ IDAY + CNC: 0
MTM for EQ Delivery: 0
Net Trading Amount: 0
Net Withdrawal Amount: 0
Start of Day Collateral: 0
Total Collateral: 0
Net Margin Available: 0


In [ ]:
# # ============================================================
# # TEMP DEBUG PARAMETERS (REMOVE LATER)
# # ============================================================

RELAX_TICKS = 1.8         # allow 1 tick relaxation
RELAX_EPS = RELAX_TICKS * TICK_SIZE

class ExecutionSimulator:
    def __init__(self, logger: TradeLogger):
        self.logger = logger

    def try_execute(
        self,
        b_hat,
        a_hat,
        bt,
        at,
        inventory: InventoryManager,
        strategy_name=""
    ):
        trade = None

        # BUY (relaxed)
        if b_hat is not None and b_hat >= at - RELAX_EPS:
            if inventory.on_buy(at):
                trade = ("BUY", at)

        # SELL (relaxed)
        elif a_hat is not None and a_hat <= bt + RELAX_EPS:
            if inventory.on_sell(bt):
                trade = ("SELL", bt)

        if trade is not None:
            side, price = trade

            self.logger.log_trade(
                strategy=strategy_name,
                side=side,
                price=price,
                bt=bt,
                at=at,
                b_hat=b_hat,
                a_hat=a_hat,
                inventory=inventory,
            )

            # Optional console print (low noise)
            print(
                f"[{strategy_name}] {side} @ {price} | "
                f"Q={inventory.Q} PnL={inventory.pnl():.2f}"
            )

        return trade
